In [5]:
!pip install ipywidgets
import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math
import torch
import torchvision
import echonet
import tqdm

In [6]:
if not os.path.exists('/workspace/data/drives/Local_SSD/sdc/Weights-20201103T193519Z-001/Weights'):
    os.system("cp -r /workspace/data/NAS/Weights-20201103T193519Z-001/ /workspace/data/drives/Local_SSD/sdc/")
#ds = echonet.datasets.Echo(root = "C:\\Datasets\\ReleasedData", split = "all")
#mean, std = echonet.utils.get_mean_and_std(ds)
mean = np.array([31.834011, 31.95879,  32.082172])
std = np.array([48.866325, 49.137333, 49.361984])
print(mean, std)

[31.834011 31.95879  32.082172] [48.866325 49.137333 49.361984]


In [7]:
def collate_fn(x):
    x, f = zip(*x)
    i = list(map(lambda t: t.shape[1], x))
    x = torch.as_tensor(np.swapaxes(np.concatenate(x, 1), 0, 1))
    return x, f, i

In [8]:
# From https://github.com/echonet/dynamic/blob/master/scripts/InitializationNotebook.ipynb
data = "Example Analysis"
outputLocation = "Example Segmentation"

segmentation_model_checkpoint = "/workspace/data/drives/Local_SSD/sdc/Weights-20201103T193519Z-001/Weights/deeplabv3_resnet50_random.pt"

# Initialize and Run Segmentation model


model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = False, aux_loss = False)
model.classifier[-1] = torch.nn.Conv2d(model.classifier[-1].in_channels, 1, kernel_size=model.classifier[-1].kernel_size)

if torch.cuda.is_available():
    print("cuda is available, original weights")
    device = torch.device("cuda")
    model = torch.nn.DataParallel(model)
    model.to(device)
    checkpoint = torch.load(segmentation_model_checkpoint)
    model.load_state_dict(checkpoint['state_dict'])
    
model.eval()

cuda is available, original weights


DataParallel(
  (module): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [10]:
import pathlib 
destinationFolder = "Example Segmentation"
duplicates_file = os.listdir(destinationFolder)
additional_duplicates = os.listdir(outputLocation)
duplicates_file = np.concatenate((duplicates_file,additional_duplicates),axis=0).tolist()

In [11]:
x = torch.arange(16).view(2, 2, 2,2)
x[0,0]

tensor([[0, 1],
        [2, 3]])

In [12]:
torch.flip(x, [3])[0,0]

tensor([[1, 0],
        [3, 2]])

In [13]:
kwargs = {"target_type": "Filename",
          "mean": mean,
          "std": std,
          "duplicates_file":duplicates_file
          
          }

test_ds = echonet.datasets.Echo(split = "external_test", external_test_location = data, length = None, period = 1, **kwargs)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size = 10, num_workers = 0, shuffle = False, pin_memory=(device.type == "cuda"), collate_fn = collate_fn)


block = 1024

if not all([os.path.isfile(os.path.join(destinationFolder, "labels", os.path.splitext(f)[0] + ".npy")) for f in test_dataloader.dataset.fnames]):
    
    pathlib.Path(os.path.join(outputLocation, "labels")).mkdir(parents = True, exist_ok = True)
        
    with torch.no_grad():
        for (x,f,i) in tqdm.tqdm(test_dataloader):
            # x = torch.flip(x,[3])
            x = x.to(device)
            y = np.concatenate([model(x[i:(i+block),:,:,:])["out"].detach().cpu().numpy() for i in range(0, x.shape[0], block)]).astype(np.float16)
    
            start = 0
        
            oldvideo = x.cpu().numpy().copy()
            oldvideo = oldvideo * std.reshape(1, 3, 1, 1)
            oldvideo = oldvideo + mean.reshape(1, 3, 1, 1)
            
            newvideo = oldvideo.copy()
            newvideo[:,2,:,:] = np.maximum(newvideo[:,2,:,:], 255. * (y[:, 0, :, :] > 0))
            
            
            for (filename, offset) in zip(f,i):
                #print(filename, offset, y[start:(start+offset), :, :, :].shape, x[start:(start+offset), :, :, :].shape)
                np.save(os.path.join(outputLocation, "labels", os.path.splitext(filename)[0]), y[start:(start+offset), 0, :, :])
                
                #plain videos
                echonet.utils.savevideo(os.path.join(outputLocation,os.path.splitext(filename)[0] + ".avi"), np.transpose(newvideo[start:(start+offset), :, :, :],(1,0,2,3)).astype(np.uint8), 50)
                #echonet.utils.savevideo(os.path.join(outputLocation,os.path.splitext(filename)[0] + "_original.avi"), np.transpose(oldvideo[start:(start+offset), :, :, :],(1,0,2,3)).astype(np.uint8), 50)
                start += offset

  0%|          | 0/1 [00:00<?, ?it/s]

removed duplicates, 0


100%|██████████| 1/1 [00:03<00:00,  3.52s/it]


In [ ]:

# fancy videos with histogram

videogeneration_ds = echonet.datasets.Echo(split="external_test", external_test_location = data, target_type=["Filename"], length=None, **kwargs, period=1, segmentation=os.path.join(destinationFolder, "labels"))
videogeneration_dataloader = torch.utils.data.DataLoader(videogeneration_ds, batch_size = 1, num_workers = 8, shuffle = False, pin_memory = False)
                

if not all([os.path.isfile(os.path.join(destinationFolder, "videos", f)) for f in videogeneration_dataloader.dataset.fnames]):
    pathlib.Path(os.path.join(destinationFolder, "fancy_videos")).mkdir(parents=True, exist_ok=True)
    pathlib.Path(os.path.join(destinationFolder, "size")).mkdir(parents=True, exist_ok=True)
    echonet.utils.latexify()
    with open(os.path.join(destinationFolder, "size.csv"), "w") as g:
        g.write("Filename,Frame,Size,ComputerSmall\n")
        for (x, filename) in tqdm.tqdm(dataloader):
            x = x.numpy()
            for i in range(len(filename)):
                img = x[i, :, :, :, :].copy()
                logit = img[2, :, :, :].copy()
                img[1, :, :, :] = img[0, :, :, :]
                img[2, :, :, :] = img[0, :, :, :]
                img = np.concatenate((img, img), 3)
                img[0, :, :, 112:] = np.maximum(255. * (logit > 0), img[0, :, :, 112:])

                img = np.concatenate((img, np.zeros_like(img)), 2)
                size = (logit > 0).sum(2).sum(1)
                try:
                    trim_min = sorted(size)[round(len(size) ** 0.05)]
                except:
                    import code; code.interact(local=dict(globals(), **locals()))
                trim_max = sorted(size)[round(len(size) ** 0.95)]
                trim_range = trim_max - trim_min
                peaks = set(scipy.signal.find_peaks(-size, distance=20, prominence=(0.50 * trim_range))[0])
                for (x, y) in enumerate(size):
                    g.write("{},{},{},{}\n".format(filename[0], x, y, 1 if x in peaks else 0))
                fig = plt.figure(figsize=(size.shape[0] / 50 * 1.5, 3))
                plt.scatter(np.arange(size.shape[0]) / 50, size, s=1)
                ylim = plt.ylim()
                for p in peaks:
                    plt.plot(np.array([p, p]) / 50, ylim, linewidth=1)
                plt.ylim(ylim)
                plt.title(os.path.splitext(filename[i])[0])
                plt.xlabel("Seconds")
                plt.ylabel("Size (pixels)")
                plt.tight_layout()
                plt.savefig(os.path.join(destinationFolder, "size", os.path.splitext(filename[i])[0] + ".pdf"))
                plt.close(fig)
                size -= size.min()
                size = size / size.max()
                size = 1 - size
                for (x, y) in enumerate(size):
                    img[:, :, int(round(115 + 100 * y)), int(round(x / len(size) * 200 + 10))] = 255.
                    interval = np.array([-3, -2, -1, 0, 1, 2, 3])
                    for a in interval:
                        for b in interval:
                            img[:, x, a + int(round(115 + 100 * y)), b + int(round(x / len(size) * 200 + 10))] = 255.
                    if x in peaks:
                        img[:, :, 200:225, b + int(round(x / len(size) * 200 + 10))] = 255.
                echonet.utils.savevideo(os.path.join(destinationFolder, "fancy_videos", filename[i]), img.astype(np.uint8), 50)       

In [ ]:

data = "C:\\Datasets\\All A4c Videos from Database Query\\DownsampledAVIs"


kwargs = {"target_type": "Filename",
          #"mean": mean,
          #"std": std,
          "length": 16,
          "period": 1
          }

test_ds = echonet.datasets.Echo(split = "external_test", external_test_location = data, **kwargs, clips="all") # unique_fnames) #
#print(test_ds.split, test_ds.fnames)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size = 1, num_workers = 5, shuffle = False, pin_memory=(False))

afew = []
anames = []

with tqdm.tqdm(total = len(test_dataloader)) as pbar:
    for i, (X, outcome) in enumerate(test_dataloader):
        print(X.shape, outcome)
        
        afew.append(X)
        anames.append(outcome)
        
        if i > 5:
            break
            
#print(afew)

outputLocation = "C:\\Datasets\\All A4c Videos from Database Query\\Segmented_DownsampledAVIs"
print(anames[0][0])
for i,a in enumerate(afew):
    a = torch.reshape(a,(-1,3,16,112,112))
    print(i,a.shape)
    print(os.path.join(outputLocation, anames[i][0]))
    echonet.utils.savevideo(os.path.join(outputLocation, anames[i][0]), a[0,:,:,:,:].numpy().astype(np.uint8), 50)

import matplotlib.pyplot as plt

a = afew[0][0,0,:,0,:,:]
a = np.transpose(a,(1,2,0))
print(a.shape)
print(torch.max(a))
#plt.show()
print(torch.min(a))
#plt.show()
#plt.imshow(afew[0][0,0,2,0,:,:])
#plt.show()
plt.imshow(a)
plt.show()